In [1]:
from dataproc import Boundary
from dataproc.storage.localfs import LocalFSStorageBackend
from dataproc.helpers import processor_name, get_processor_by_name, get_processor_meta_by_name, list_processors


In [2]:
iso3 = 'bgd'


In [3]:
storage_backend = LocalFSStorageBackend('./tmp')


In [4]:
list_processors()


/home/mert2014/projects/infra-risk-vis/irv-autopkg/dataproc/processors/core/__init__.py:11: UserWarning: failed to load module isimp_drought.version_1 due to cannot import name 'output_filename' from 'dataproc.helpers' (/home/mert2014/projects/infra-risk-vis/irv-autopkg/dataproc/helpers.py)
  warnings.warn(f"failed to load module {module_name} due to {err}")


defaultdict(list,
            {'gri_osm': ['roads_and_rail_version_1'],
             'gridfinder': ['version_1'],
             'jrc_ghsl_built_c': ['r2022_epoch2018_10m_mszfun'],
             'jrc_ghsl_population': ['r2022_epoch2020_1km'],
             'natural_earth_raster': ['version_1'],
             'natural_earth_vector': ['version_1'],
             'storm': ['global_mosaics_version_1'],
             'wri_aqueduct': ['version_2'],
             'wri_powerplants': ['version_130']})

In [5]:
boundary_name = iso3
baseurl = "https://irv-autopkg.s3.eu-west-2.amazonaws.com"

## TODO figure out base datapackage (basic is almost empty dict)
base_datapackage = {}

output_processors = []
for dataset in storage_backend.package_datasets(boundary_name):
    try:
        for version in storage_backend.dataset_versions(boundary_name, dataset):
            proc_name = processor_name(dataset, version)
            print(f"{boundary_name=}: {proc_name=} {dataset=} {version=}")
            meta_cls = get_processor_meta_by_name(proc_name)
            assert meta_cls is not None, 'Invalid Metadata class'
            metadata = meta_cls()
            ds_version_uri = f"{baseurl}/{boundary_name}/datasets/{dataset}/{version}/"
            processor_cls = get_processor_by_name(proc_name)
            assert processor_cls is not None, 'Invalid Processor class'
            processor = processor_cls(
                metadata=metadata,
                storage_backend=storage_backend,
                boundary=Boundary(iso3, {}, {}),
                task_executor=None,
                processing_root_folder='root'
            )

    except Exception as e:
        print(f"Error {boundary_name=}: {proc_name=} {dataset=} {version=}")
        print(e)
    ## TODO first run over bgd locally
    ## TODO then on SoGE cluster, run over all countries and datasets
    break


boundary_name='bgd': proc_name='gri_osm.roads_and_rail_version_1' dataset='gri_osm' version='roads_and_rail_version_1'


In [6]:
processor.generate_datapackage_resource()


root/tmp/bgd/gri_osm-roads_and_rail_version_1-bgd.gpkg: No such file or directory
40575486787F0000:error:80000002:system library:file_ctrl:No such file or directory:crypto/bio/bss_file.c:297:calling fopen(root/tmp/bgd/gri_osm-roads_and_rail_version_1-bgd.gpkg, r)
40575486787F0000:error:10080002:BIO routines:file_ctrl:system lib:crypto/bio/bss_file.c:300:


CalledProcessError: Command '['openssl', 'sha1', 'root/tmp/bgd/gri_osm-roads_and_rail_version_1-bgd.gpkg']' returned non-zero exit status 1.